In [3]:
!python -m pip install matplotlib numpy

You are using pip version 10.0.1, however version 21.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [4]:
from sys import float_info

import numpy as np
from numpy.linalg import norm, pinv, solve

def ordinary_least_squares(x: np.ndarray, y: np.ndarray, order: int,
        robust: bool = False, delta: float = 1.0, epsilon: float = 1E-7,
        max_iter: int = 100) -> np.ndarray:
    if (not isinstance(x, np.ndarray) or not isinstance(y, np.ndarray)
            or not isinstance(order, int) or not isinstance(robust, bool)
            or not isinstance(delta, float) or not isinstance(epsilon, float)
            or not isinstance(max_iter, int)):
        raise TypeError()
    if (len(x.shape) != 1 or x.shape != y.shape or order < 1
            or delta < float_info.epsilon or epsilon < float_info.epsilon
            or max_iter < 1):
        raise ValueError()

    m = order + 1
    x_upper = np.vander(x, m, increasing=False)
    left, right = x_upper.T @ x_upper, x_upper.T @ y
    print('Condition number =', np.linalg.cond(left))

    beta = solve(left, right)
    if not robust:
        return beta

    for _ in range (max_iter):
        np.fill_diagonal(w_upper, 1.0 / np.abs(beta))
        next_beta = np.linalg.solve(left + alpha * w_upper, right)
        error = norm(next_beta - beta, ord=2)
        beta, next_beta = next_beta, beta
        if (error < epsilon):
            break
    return beta

In [ ]:
def f(x: float) -> float:
    return x**2 / 5.0 - 3.0 * x + 5.0

start, stop = -10, 10
x = np.linspace(start, stop, 20)
x_extended = np.linspace(start, stop, 2 ** 20)
f_vectorize = np.vectorize(f)
y = f_vectorize(x)

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(16, 9), dpi=400)
subplot = fig.add_subplot(111, facecolor='#FFFFFF')
subplot.plot(x, y, color='red', lw=2, label='y')
plt.legend()
plt.show()

In [ ]:
from numpy.random import normal

y_noise = y + normal(loc=0, scale=3.0, size=len(y))
fig = plt.figure(figsize=(16, 9), dpi=400)
subplot = fig.add_subplot(111, facecolor='#FFFFFF')
subplot.plot(x, y_noise, 'Xr', label='y_noise')
subplot.plot(x, y, color='green', lw=2, label='y')
plt.legend()
plt.show()

In [ ]:
poly = ordinary_least_squares(x, y_noise, order=2, p=0, alpha=0.0)
polyval = np.polyval(poly, x_extended)
print(poly)

fig = plt.figure(figsize=(16, 9), dpi=400)
subplot = fig.add_subplot(111, facecolor='#FFFFFF')
subplot.plot(x, y_noise, 'Xr', label='y_noise')
subplot.plot(x, y, color='green', lw=2, label='y')
subplot.plot(x_extended, polyval, color='blue', lw=2, label='polyval')
plt.legend()
plt.show()

In [ ]:
poly = ordinary_least_squares(x, y_noise, order=4, p=0, alpha=0.0)
polyval = np.polyval(poly, x_extended)
print(poly)

fig = plt.figure(figsize=(16, 9), dpi=400)
subplot = fig.add_subplot(111, facecolor='#FFFFFF')
subplot.plot(x, y_noise, 'Xr', label='y_noise')
subplot.plot(x, y, color='green', lw=2, label='y')
subplot.plot(x_extended, polyval, color='blue', lw=2, label='polyval')
plt.legend()
plt.show()

In [ ]:
COLORS = ['blue', 'brown', 'cyan', 'gray', 'olive', 'orange', 'pink', 'purple']

fig = plt.figure(figsize=(16, 9), dpi=400)
subplot = fig.add_subplot(111, facecolor='#FFFFFF')
subplot.plot(x, y_noise, 'Xr', label='y_noise')
subplot.plot(x, y, color='green', lw=2, label='y')

for i in range(len(COLORS)):
    alpha = 2.0 ** (i - 6)
    poly = ordinary_least_squares(x, y_noise, order=4, p=2, alpha=alpha)
    polyval = np.polyval(poly, x_extended)
    label = 'polyval: alpha = ' + str(alpha)
    subplot.plot(x_extended, polyval, color=COLORS[i], lw=2, label=label)
    print(poly)
plt.legend()
plt.show()

In [ ]:
fig = plt.figure(figsize=(16, 9), dpi=400)
subplot = fig.add_subplot(111, facecolor='#FFFFFF')
subplot.plot(x, y_noise, 'Xr', label='y_noise')
subplot.plot(x, y, color='green', lw=2, label='y')

poly_prev = None
index = 0
for i in range(30):
    rcond = 1.5 ** (i - 25)
    poly = ordinary_least_squares_svd(x, y_noise, order=4, rcond=rcond)
    if (poly == poly_prev).all():
        continue
    poly_prev = poly
    polyval = np.polyval(poly, x_extended)
    label = 'polyval: rcond = ' + str(rcond)
    subplot.plot(x_extended, polyval, color=COLORS[index], lw=2, label=label)
    index += 1
    print(poly)
plt.legend()
plt.show()